# Projeto Prático Perceptron Rosenblatt

Neste projeto prático, o objetivo é implementar o algoritmo de treinamento mediante Aprendizado Supervisionado 
do neurônio Perceptron de Rosenblatt aplicado em problemas de classificação.

## Equipe:

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
3. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
4. Lilian Iazzai De Souza Oliveira | lilianiazzai - 2215080018
5. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

In [1]:
# identificador do arquivo
id = (7+9+6+8+5)%4
print(id)

3


In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm
import prettytable

In [3]:
def get_ndarray_from_file(filename: str) -> np.ndarray:
    """ Formato dos dados -> (m, 1, 3) : [[[x_1, x_2, y_d]], ...]"""
    return np.fromfile(filename, dtype=np.float64).reshape(-1, 1, 3)


data = get_ndarray_from_file(f"./data/data{id}.txt")
print(data.shape, "\n", data[0])

(600, 1, 3) 
 [[ 0.06162809 -0.1242566   0.        ]]
